In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('./mnist/data/',one_hot=True)

C:\Users\Owner\Anaconda3\envs\vc\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Owner\Anaconda3\envs\vc\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Owner\Anaconda3\envs\vc\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Owner\Anaconda3\envs\vc\lib\site-packages\tensorflow\python\framework\dty

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# 신경망 모델 구성

global_step = tf.Variable(0, trainable=False, name='global_step')

X=tf.placeholder(tf.float32,[None,28,28,1])
Y=tf.placeholder(tf.float32,[None,10])
keep_prob=tf.placeholder(tf.float32)

'''
W1 [3 3 1 32] -> [3 3]: 커널 크기, 1: 입력값 X 의 특성수, 32: 필터 갯수
L1 Conv shape=(?, 28, 28, 32)
Pool     ->(?, 14, 14, 32)
'''
# output_size=(W-F+2P)/S+1
W1=tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
# 이제 필터가 돌아다니는거임/ conv2d를 통과해도 28 * 28 크기를 가짐, 32개의 필터임으로 32개의 feature map이 생김
L1=tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding='SAME')
L1=tf.nn.relu(L1)
# Pooling 28*28->14*14
# ksize [1,2,2,1] 2칸씩 이동하면서 결과 1개 / 4칸당 가장 큰 것 / 계산값이 나누어 떨어지도록 padding option 추가
L1=tf.nn.max_pool(L1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

'''
L2 Conv shape=(?, 14, 14, 64)
Pool     ->(?, 7, 7, 64)
W2 의 [3, 3, 32, 64] 에서 32 는 L1 에서 출력된 W1 의 마지막 차원, 필터의 크기 / 64 : 필터 갯수
'''

W2=tf.Variable(tf.random_normal([3,3,32,64],stddev=0.01))
L2=tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding='SAME')
L2=tf.nn.relu(L2)
L2=tf.nn.max_pool(L2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

'''
FC 레이어 : 입력값 7x7x64 / 256

'''
W3=tf.Variable(tf.random_normal([7*7*64,256],stddev=0.01))
# [1,7*7*64]
L3=tf.reshape(L2,[-1,7*7*64])
L3=tf.matmul(L3,W3)
L3=tf.nn.relu(L3)
# [1,256]
L3=tf.nn.dropout(L3,keep_prob)

# 256을 입력으로 받아 10으로 OUTPUT
W4=tf.Variable(tf.random_normal([256,10],stddev=0.01))
model=tf.matmul(L3,W4)

cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))

optimizer=tf.train.RMSPropOptimizer(0.001,0.9).minimize(cost,global_step=global_step)

# 신경망 모델 학습
with tf.Session() as sess:
    
    # 모델 저장
    saver=tf.train.Saver(tf.global_variables())
    ckpt=tf.train.get_checkpoint_state('./model')
    if ckpt and  tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
        print("저장된 checkpoint가 있습니다.")
    
        # 마지막 파일 명에서 저장된 STEP 받아오기
        global_step = int(ckpt.model_checkpoint_path.split('-')[1].split('.')[0])
        saver.restore(sess,ckpt.model_checkpoint_path)
        #saver.restore(sess, tf.train.latest_checkpoint('./model') )
    else:
        print("저장된 checkpoint가 없습니다.")
        sess.run(tf.global_variables_initializer())
        global_step=0
    
    batch_size=100
    total_batch=int(mnist.train.num_examples/batch_size)
    
    for epoch in range(global_step,6):
        total_cost=0
        
        for i in range(total_batch):
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            # 이미지 데이터 cnn모델을 위한 자료형태인 [28 28 1]의 형태로 재구성합니다.
            batch_xs=batch_xs.reshape(-1,28,28,1)
            _,cost_val=sess.run([optimizer,cost],feed_dict={X:batch_xs, Y:batch_ys, keep_prob:0.7})
            
            total_cost+=cost_val
            
        print('Epoch:', '%04d' % (epoch + 1),
          'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))
        
        #if(epoch)%3==0:
        print("모델 저장")
        saver.save(sess,"./model/mnist_cnn",global_step=epoch)
        
    print('최적화 완료')
    
    
    # 결과 확인
    is_correct=tf.equal(tf.argmax(model,1),tf.argmax(Y,1))
    accuracy=tf.reduce_mean(tf.cast(is_correct,tf.float32))
    print('정확도 : ',sess.run(accuracy,feed_dict={X:mnist.test.images.reshape(-1,28,28,1),
                                               Y:mnist.test.labels, keep_prob:1}))
            






저장된 checkpoint가 있습니다.
INFO:tensorflow:Restoring parameters from ./model\mnist_cnn-3
Epoch: 0004 Avg. cost = 0.041
모델 저장
Epoch: 0005 Avg. cost = 0.035
모델 저장
Epoch: 0006 Avg. cost = 0.030
모델 저장
최적화 완료
정확도 :  0.9899
